Karol Janic

kwiecień 2023

zad2.ipynb - Znajdowanie najkrótszej ścieżki pomiędzy miastami z ograniczeniem czasowym.

In [1]:
using JuMP
using GLPK

using JSON

In [2]:
dane_wejsciowe = JSON.parse(read("data2.json", String))

miasta = dane_wejsciowe["miasta"]
polaczenia = dane_wejsciowe["polaczenia"]
lokalizacja_poczatkowa = dane_wejsciowe["lokalizacja_poczatkowa"]
lokalizacja_koncowa = dane_wejsciowe["lokalizacja_koncowa"]
ograniczenie_czasowe = dane_wejsciowe["ograniczenie_czasowe"]

440

In [3]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

Definicja zmiennych decyzyjnych(informacja czy przejeżdzamy z jednego do drugiego miasta bezpośrednio) oraz ograniczenie ich wartości do 0 lub 1:

In [4]:
@variable(model, x[miasta, miasta], Bin)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Any["Warszawa", "Krakow", "Lodz", "Wroclaw", "Poznan", "Gdansk", "Gdynia", "Szczecin", "Torun", "Bydgoszcz"]
    Dimension 2, Any["Warszawa", "Krakow", "Lodz", "Wroclaw", "Poznan", "Gdansk", "Gdynia", "Szczecin", "Torun", "Bydgoszcz"]
And data, a 10×10 Matrix{VariableRef}:
 x[Warszawa,Warszawa]   x[Warszawa,Krakow]   …  x[Warszawa,Bydgoszcz]
 x[Krakow,Warszawa]     x[Krakow,Krakow]        x[Krakow,Bydgoszcz]
 x[Lodz,Warszawa]       x[Lodz,Krakow]          x[Lodz,Bydgoszcz]
 x[Wroclaw,Warszawa]    x[Wroclaw,Krakow]       x[Wroclaw,Bydgoszcz]
 x[Poznan,Warszawa]     x[Poznan,Krakow]        x[Poznan,Bydgoszcz]
 x[Gdansk,Warszawa]     x[Gdansk,Krakow]     …  x[Gdansk,Bydgoszcz]
 x[Gdynia,Warszawa]     x[Gdynia,Krakow]        x[Gdynia,Bydgoszcz]
 x[Szczecin,Warszawa]   x[Szczecin,Krakow]      x[Szczecin,Bydgoszcz]
 x[Torun,Warszawa]      x[Torun,Krakow]         x[Torun,Bydgoszcz]
 x[Bydgoszcz,Warszawa]  x[Bydg

Ograniczenie możliwości przejazdu pomiędzy miastami, których nie łączy droga:

In [5]:
for miasto1 in miasta, miasto2 in miasta
    if !(miasto2 in keys(polaczenia[miasto1]))
        @constraint(model, x[miasto1, miasto2] == 0)
    end
end

Ograniczenie zmiennych aby zaznaczyć miejscowość początkową i końcową oraz zachować ciągłość trasy:

In [6]:
for miasto in miasta
    if miasto == lokalizacja_poczatkowa
        @constraint(model, sum(x[miasto,:]) - sum(x[:,miasto]) == 1)
    elseif  miasto == lokalizacja_koncowa
        @constraint(model, sum(x[miasto,:]) - sum(x[:,miasto]) == -1)
    else
        @constraint(model, sum(x[miasto,:]) - sum(x[:,miasto]) == 0)
    end
end

Ograniczenie na czas przejazdu w minutach(suma czasów przejazdów po drogach w minutach, które znajdują się na trasie):

In [7]:
@constraint(model, 
    sum(x[miasto1, miasto2] * polaczenia[miasto1][miasto2]["czas"] for miasto1 in miasta, miasto2 in keys(polaczenia[miasto1])) <= ograniczenie_czasowe)

217 x[Krakow,Warszawa] + 104 x[Lodz,Warszawa] + 229 x[Wroclaw,Warszawa] + 199 x[Poznan,Warszawa] + 240 x[Gdansk,Warszawa] + 264 x[Gdynia,Warszawa] + 170 x[Torun,Warszawa] + 205 x[Bydgoszcz,Warszawa] + 199 x[Lodz,Krakow] + 189 x[Wroclaw,Krakow] + 300 x[Poznan,Krakow] + 387 x[Szczecin,Krakow] + 102 x[Warszawa,Lodz] + 277 x[Krakow,Lodz] + 147 x[Wroclaw,Lodz] + 150 x[Poznan,Lodz] + 207 x[Gdansk,Lodz] + 159 x[Szczecin,Lodz] + 114 x[Bydgoszcz,Lodz] + 232 x[Warszawa,Wroclaw] + 206 x[Krakow,Wroclaw] + 145 x[Lodz,Wroclaw] + 134 x[Poznan,Wroclaw] + 319 x[Gdansk,Wroclaw] + 313 x[Gdynia,Wroclaw] + 253 x[Szczecin,Wroclaw] + 249 x[Torun,Wroclaw] + 212 x[Bydgoszcz,Wroclaw] + 211 x[Warszawa,Poznan] + 312 x[Krakow,Poznan] + 168 x[Lodz,Poznan] + 134 x[Wroclaw,Poznan] + 205 x[Gdansk,Poznan] + 220 x[Gdynia,Poznan] + 177 x[Szczecin,Poznan] + 146 x[Torun,Poznan] + 104 x[Bydgoszcz,Poznan] + 224 x[Warszawa,Gdansk] + 207 x[Lodz,Gdansk] + 194 x[Poznan,Gdansk] + 30 x[Gdynia,Gdansk] + 266 x[Szczecin,Gdansk] + 109

Funkcja kosztu w kilometrach(suma długości dróg w kilomentrach, które znajdują się na trasie):

In [8]:
@objective(model, Min, 
    sum(x[miasto1, miasto2] * polaczenia[miasto1][miasto2]["odleglosc"] for miasto1 in miasta, miasto2 in keys(polaczenia[miasto1])))


340 x[Warszawa,Gdansk] + 308 x[Warszawa,Bydgoszcz] + 259 x[Warszawa,Torun] + 348 x[Warszawa,Wroclaw] + 136 x[Warszawa,Lodz] + 312 x[Warszawa,Poznan] + 566 x[Warszawa,Szczecin] + 495 x[Krakow,Bydgoszcz] + 446 x[Krakow,Torun] + 268 x[Krakow,Wroclaw] + 434 x[Krakow,Lodz] + 459 x[Krakow,Poznan] + 648 x[Krakow,Szczecin] + 290 x[Krakow,Warszawa] + 362 x[Lodz,Gdynia] + 340 x[Lodz,Gdansk] + 183 x[Lodz,Torun] + 280 x[Lodz,Krakow] + 214 x[Lodz,Wroclaw] + 211 x[Lodz,Poznan] + 140 x[Lodz,Warszawa] + 509 x[Wroclaw,Gdynia] + 358 x[Wroclaw,Torun] + 273 x[Wroclaw,Krakow] + 222 x[Wroclaw,Lodz] + 182 x[Wroclaw,Poznan] + 417 x[Wroclaw,Szczecin] + 355 x[Wroclaw,Warszawa] + 332 x[Poznan,Gdynia] + 311 x[Poznan,Gdansk] + 139 x[Poznan,Bydgoszcz] + 458 x[Poznan,Krakow] + 185 x[Poznan,Wroclaw] + 218 x[Poznan,Lodz] + 311 x[Poznan,Warszawa] + 20 x[Gdansk,Gdynia] + 167 x[Gdansk,Bydgoszcz] + 170 x[Gdansk,Torun] + 482 x[Gdansk,Wroclaw] + 337 x[Gdansk,Lodz] + 311 x[Gdansk,Poznan] + 348 x[Gdansk,Warszawa] + 36 x[Gdyni

In [9]:
optimize!(model)

In [10]:
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 6.53000e+02
  Objective bound    : 6.53000e+02
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 6.58512e-03


In [11]:
JuMP.objective_value(model)

653.0

In [12]:
total_time = 0
for miasto1 in miasta
    for miasto2 in keys(polaczenia[miasto1])
        total_time += value(x[miasto1, miasto2]) * polaczenia[miasto1][miasto2]["czas"]
    end
end

println("Total time: ", total_time)

Total time: 436.0


In [13]:
for miasto1 in miasta, miasto2 in miasta
    if value(x[miasto1, miasto2]) > 0.0
        println(miasto1, " => ", miasto2, "   =", value(x[miasto1, miasto2]))
    end
end

Lodz => Krakow   =1.0
Gdansk => Lodz   =1.0
Gdynia => Gdansk   =1.0
